# XtEHR Logical model processing

* ophalen XtEHR Logical Models
* consistente layout met zibs
* klaar voor een mapping - zie mapping Notebook

## Relevante kern content

NL kernset:
- Behandelaanwijzing

ePS required:
- Medicatie
- Diagnose/ Probleemlijst(actief)
- Verrichtingen(<6mnd, ≥ 6mnd)
- Allergieën en intoleranties + - - 
- Alerts
- Hulpmiddel
- Patiënt

ePS recommended + optional:
- Vaccinations
- Implants
- Treatment restrictions
- Vital signs
- History of illness
- Pregnancy
- Social history
- Functional status
- Results
- Plan of Care
- Encounters

# Imports

In [23]:
import pandas as pd
import requests
import json
import os
from io import StringIO

# Download JSON

* Dit hoeft maar een keer te gebeuren, bij nieuwe mappings download de JSON die nodig is. 
* Kan ook door directe download 'raw' JSON.
* Dit zijn de XtEHR Logical Models

In [24]:
xtehrlist = ['Observation',  'Specimen', 'ServiceRequest','LaboratoryReport']
xtehrbase = 'https://build.fhir.org/ig/Xt-EHR/xt-ehr-common/StructureDefinition-EHDS'
if (not os.path.exists('xtehr')):
    os.makedirs('xtehr')
for i, name in enumerate(xtehrlist):
    print(i, name)  # prints one line per iteration
    url = f"{xtehrbase}{name}.json"
    r = requests.get(url)
    r.raise_for_status()
    with open(os.path.join('xtehr', 'downloads', f'{name}.json'), 'w', encoding='utf-8') as f:
        json.dump(r.json(), f, indent=4)

0 Observation
1 Specimen
2 ServiceRequest
3 LaboratoryReport


# Kies en lees het te verwerken Logical Model

* Toon en verifieer dat de uitvoer er is

In [25]:
base = 'xtehr'
file = xtehrlist[3]
url = os.path.join(base, 'downloads', file + '.json')
with open(url) as jsonfile:
    xlm = json.load(jsonfile)['snapshot']['element']
dfx = pd.DataFrame(xlm)
dfx

,id,path,short,definition,min,max,base,isModifier,mapping,type,binding
0,EHDSLaboratoryReport,EHDSLaboratoryReport,Laboratory report model,EHDS refined base model for Laboratory result ...,0,*,"{'path': 'Base', 'min': 0, 'max': '*'}",False,"[{'identity': 'rim', 'map': 'n/a'}]",NaN,NaN
1,EHDSLaboratoryReport.header,EHDSLaboratoryReport.header,Laboratory Report header,Common header for all patient-related data,1,1,"{'path': 'EHDSDataSet.header', 'min': 1, 'max'...",NaN,NaN,[{'code': 'Base'}],NaN
2,EHDSLaboratoryReport.header.subject,EHDSLaboratoryReport.header.subject,Subject,Patient/subject information,1,1,"{'path': 'EHDSDataSet.header.subject', 'min': ...",NaN,NaN,[{'code': 'https://www.xt-ehr.eu/specification...,NaN
3,EHDSLaboratoryReport.header.identifier,EHDSLaboratoryReport.header.identifier,Document ID,Unique identifier of the document,1,*,"{'path': 'EHDSDataSet.header.identifier', 'min...",NaN,NaN,[{'code': 'Identifier'}],NaN
4,EHDSLaboratoryReport.header.authorship,EHDSLaboratoryReport.header.authorship,Authorship,Resource authoring details,1,*,"{'path': 'EHDSDataSet.header.authorship', 'min...",NaN,NaN,[{'code': 'Base'}],NaN
5,EHDSLaboratoryReport.header.authorship.author[x],EHDSLaboratoryReport.header.authorship.author[x],Author,Author(s) by whom the resource was/were author...,1,1,{'path': 'EHDSDataSet.header.authorship.author...,NaN,NaN,[{'code': 'https://www.xt-ehr.eu/specification...,NaN
6,EHDSLaboratoryReport.header.authorship.datetime,EHDSLaboratoryReport.header.authorship.datetime,Date and time of authoring/issuing,Date and time of the issuing the document/reso...,1,1,{'path': 'EHDSDataSet.header.authorship.dateti...,NaN,NaN,[{'code': 'dateTime'}],NaN
7,EHDSLaboratoryReport.header.lastUpdate,EHDSLaboratoryReport.header.lastUpdate,Date and time of the last update to the resource,Date and time of the last update to the docume...,0,1,"{'path': 'EHDSDataSet.header.lastUpdate', 'min...",NaN,NaN,[{'code': 'dateTime'}],NaN
8,EHDSLaboratoryReport.header.status,EHDSLaboratoryReport.header.status,Status of the resource,Status of the resource,1,1,"{'path': 'EHDSDataSet.header.status', 'min': 1...",NaN,NaN,[{'code': 'CodeableConcept'}],NaN
9,EHDSLaboratoryReport.header.statusReason[x],EHDSLaboratoryReport.header.statusReason[x],Reason for the current status of the resource.,Reason for the current status of the resource.,0,1,"{'path': 'EHDSDataSet.header.statusReason[x]',...",NaN,NaN,"[{'code': 'CodeableConcept'}, {'code': 'string'}]",NaN


# Opschonen uitvoer zodat zib en xtehr consistent zijn

* een card. veld met n..m
* type (is een JSON object) omzetten naar tekst
* alleen nodige kolommen houden

In [26]:
dfx.insert(2, 'card.', None)
dfx['card.'] = dfx['min'].astype(str) + '..' + dfx['max'].astype(str)
for i, row in dfx.iterrows():
    if 'type' in row:
        # row['type'][0]['code']
        try:
            dfx.at[i, 'type'] = row['type'][0]['code'].split('/')[-1]
        except TypeError:
            pass
dfx= dfx[['id', 'path', 'short', 'definition', 'type', 'card.', 'binding']]
dfx

,id,path,short,definition,type,card.,binding
0,EHDSLaboratoryReport,EHDSLaboratoryReport,Laboratory report model,EHDS refined base model for Laboratory result ...,NaN,0..*,NaN
1,EHDSLaboratoryReport.header,EHDSLaboratoryReport.header,Laboratory Report header,Common header for all patient-related data,Base,1..1,NaN
2,EHDSLaboratoryReport.header.subject,EHDSLaboratoryReport.header.subject,Subject,Patient/subject information,EHDSPatient,1..1,NaN
3,EHDSLaboratoryReport.header.identifier,EHDSLaboratoryReport.header.identifier,Document ID,Unique identifier of the document,Identifier,1..*,NaN
4,EHDSLaboratoryReport.header.authorship,EHDSLaboratoryReport.header.authorship,Authorship,Resource authoring details,Base,1..*,NaN
5,EHDSLaboratoryReport.header.authorship.author[x],EHDSLaboratoryReport.header.authorship.author[x],Author,Author(s) by whom the resource was/were author...,EHDSHealthProfessional,1..1,NaN
6,EHDSLaboratoryReport.header.authorship.datetime,EHDSLaboratoryReport.header.authorship.datetime,Date and time of authoring/issuing,Date and time of the issuing the document/reso...,dateTime,1..1,NaN
7,EHDSLaboratoryReport.header.lastUpdate,EHDSLaboratoryReport.header.lastUpdate,Date and time of the last update to the resource,Date and time of the last update to the docume...,dateTime,0..1,NaN
8,EHDSLaboratoryReport.header.status,EHDSLaboratoryReport.header.status,Status of the resource,Status of the resource,CodeableConcept,1..1,NaN
9,EHDSLaboratoryReport.header.statusReason[x],EHDSLaboratoryReport.header.statusReason[x],Reason for the current status of the resource.,Reason for the current status of the resource.,CodeableConcept,0..1,NaN


# Opslaan in Excel

In [27]:
url = os.path.join(base, 'mappable', file + '.xlsx')
dfx.to_excel(url, index=False)